# Claude + E2B Solana Development Environment

**Powered by [Code Interpreter SDK](https://github.com/e2b-dev/code-interpreter) by [E2B](https://e2b.dev/docs)**

This notebook creates a secure Solana development environment using Claude and E2B's code interpreter.

In [ ]:
%pip install e2b_code_interpreter==1.0.0 anthropic==0.35.0 python-dotenv==1.0.1

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# API keys
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
E2B_API_KEY = os.getenv("E2B_API_KEY")

# Use Claude 3.5 Sonnet for best results
MODEL_NAME = 'claude-3-5-sonnet-20241022'

SYSTEM_PROMPT = """
## Your role
You are a Solana blockchain development expert. Your job is to help the user with Solana development tasks 
using Python and the Solana toolchain. You can run Python code directly in the notebook environment.

## Environment capabilities
- You can install and use any Python packages related to Solana development
- You can create and manipulate Solana wallets, tokens, and transactions
- You have access to the internet for documentation and API requests
- You can work with local files for storing keypairs and other data

## Task approach
1. When asked to perform a Solana-related task, break it down into clear steps
2. Implement and test using the Python Solana libraries
3. Explain what each part of your code does in a clear, educational way
4. Handle errors gracefully and explain solutions

## Solana developer guidelines
- Always use best security practices when working with private keys
- Help users understand Solana's programming model and concepts
- Suggest optimal approaches based on Solana's specific architecture
- Don't hesitate to suggest improvements to the user's approach if you see better alternatives
"""

tools = [
    {
        "name": "execute_python",
        "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
        "input_schema": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The python code to execute in a single cell."
                }
            },
            "required": ["code"]
        }
    }
]

In [ ]:
def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.run_code(code,
  on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
  on_stdout=lambda stdout: print("[Code Interpreter]", stdout))

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec.results

In [ ]:
from anthropic import Anthropic
client = Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

def process_tool_call(e2b_code_interpreter, tool_name, tool_input):
    if tool_name == "execute_python":
        return code_interpret(e2b_code_interpreter, tool_input["code"])
    return []

def chat_with_claude(e2b_code_interpreter, user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    message = client.messages.create(
        model=MODEL_NAME,
        system=SYSTEM_PROMPT,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        code_interpreter_results = process_tool_call(e2b_code_interpreter, tool_name, tool_input)

        print(f"Tool Result: {code_interpreter_results}")
        return code_interpreter_results

In [ ]:
from e2b_code_interpreter import CodeInterpreter

e2b_code_interpreter = CodeInterpreter(api_key=E2B_API_KEY)

# Setup the Solana development environment
setup_message = """Set up a Solana development environment in this notebook. Please:
1. Install the necessary Python packages for Solana development
2. Create a helper function to generate a new Solana wallet
3. Show how to check the balance of a wallet
4. Demonstrate a simple transaction
"""

chat_with_claude(e2b_code_interpreter, setup_message)

In [ ]:
# Working with the CSV airdrop file
airdrop_message = """I have a CSV file for a Solana airdrop with this format:
```
8qVCcf8noS4m8z87Hk5nsCSiRj6VU31MNGaU5VhSBBkz,, 1000000
HTFmbL1uL9UHBjUHgVEtzi7mKuiA7HAsgaCpNdQ7jW3f,, 1000000
8mPLiSBnYV4Cd2HxXrTSxbWBRdWEKQyzbMEFxWpHDQbq,, 1000000
```

There seems to be an issue with the formatting. Can you:
1. Create a function to parse and validate this CSV format for Solana airdrops
2. Fix any formatting issues
3. Show how we would use this data to perform token airdrops on Solana
"""

chat_with_claude(e2b_code_interpreter, airdrop_message)

In [ ]:
# Ask more Solana development questions here
custom_message = """How can I create an SPL token on Solana using Python? Please show me example code that:
1. Creates a new SPL token with custom parameters (name, symbol, decimals)
2. Mints tokens to an owner wallet
3. Transfers tokens to another wallet
"""

chat_with_claude(e2b_code_interpreter, custom_message)

In [ ]:
# Clean up the E2B code interpreter when done
e2b_code_interpreter.close()